# Deepfake detector

In [3]:
from PIL import Image
import os
import torch
from torch.utils.data import TensorDataset, DataLoader, ConcatDataset
from torchvision import transforms

def load_images_with_labels(folder, label):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            with Image.open(img_path) as img:
                img_copy = img.copy()
                images.append(img_copy)
                labels.append(label)
        except Exception as e:
            print(f"Errore nell'aprire {img_path}: {e}")
    return images, labels

In [4]:
fake_train_path = 'archive/train/FAKE'
real_train_path = 'archive/train/REAL'
fake_test_path = 'archive/test/FAKE'
real_test_path = 'archive/test/REAL'

# TRAIN
fake_train_images, fake_train_labels = load_images_with_labels(fake_train_path, 0)
real_train_images, real_train_labels = load_images_with_labels(real_train_path, 1)

# TEST
fake_test_images, fake_test_labels = load_images_with_labels(fake_test_path, 0)
real_test_images, real_test_labels = load_images_with_labels(real_test_path, 1)

In [5]:
transform = transforms.Compose([
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])

# Tensors
fake_train_tensors = [transform(image) for image in fake_train_images]
real_train_tensors = [transform(image) for image in real_train_images]
fake_test_tensors = [transform(image) for image in fake_test_images]
real_test_tensors = [transform(image) for image in real_test_images]

fake_train_labels_tensor = torch.tensor(fake_train_labels, dtype=torch.long)
real_train_labels_tensor = torch.tensor(real_train_labels, dtype=torch.long)
fake_test_labels_tensor = torch.tensor(fake_test_labels, dtype=torch.long)
real_test_labels_tensor = torch.tensor(real_test_labels, dtype=torch.long)

In [6]:
# TRAIN
fake_train_dataset = TensorDataset(torch.stack(fake_train_tensors), fake_train_labels_tensor)
real_train_dataset = TensorDataset(torch.stack(real_train_tensors), real_train_labels_tensor)
train_dataset = ConcatDataset([fake_train_dataset, real_train_dataset])
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#  TEST
fake_test_dataset = TensorDataset(torch.stack(fake_test_tensors), fake_test_labels_tensor)
real_test_dataset = TensorDataset(torch.stack(real_test_tensors), real_test_labels_tensor)
test_dataset = ConcatDataset([fake_test_dataset, real_test_dataset])
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)